#  Homework 8 - Berkeley STAT 157

**Your name: XX, SID YY, teammates A,B,C** (Please add your name, SID and teammates to ease Ryan and Rachel to grade.)

**Please submit your homework through [gradescope](http://gradescope.com/)**

Handout 4/9/2019, due 4/16/2019 by 4pm.

This homework deals with sequence models for text and numbers. Due to the computational cost, we strongly encourage you to implement this on a GPU enabled machine. To make things a bit more interesting we will use a larger text collection here - [Shakespeare's collected works](http://www.gutenberg.org/files/100/100-0.txt) which are freely downloadable at Project Gutenberg. 

**This is teamwork.**

## Prerequisites - Load Data

In [38]:
import urllib3
import collections
import re
shakespeare = 'http://www.gutenberg.org/files/100/100-0.txt'

http = urllib3.PoolManager()
text = http.request('GET', shakespeare).data.decode('utf-8')
raw_dataset = ' '.join(re.sub('[^A-Za-z]+', ' ', text).lower().split())

print('number of characters: ', len(raw_dataset))
print(raw_dataset[0:70])

number of characters:  5032359
project gutenberg s the complete works of william shakespeare by willi


This dataset is quite a bit bigger than the time machine (5 million vs. 160k). For convenience we also include the remaining preprocessing steps. A bigger dataset will allow us to generate more meaningful models. 

In [39]:
idx_to_char = list(set(raw_dataset))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
corpus_indices = [char_to_idx[char] for char in raw_dataset]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

train_indices = corpus_indices[:-100000]
test_indices = corpus_indices[-100000:]

chars: project gutenberg s 
indices: [20, 6, 7, 18, 14, 12, 15, 9, 11, 13, 15, 14, 3, 4, 14, 6, 11, 9, 0, 9]


In [40]:
print(vocab_size)
print(idx_to_char)
print(char_to_idx)
print(len(corpus_indices))

27
['s', 'i', 'l', 'n', 'b', 'v', 'r', 'o', 'd', ' ', 'q', 'g', 'c', 'u', 'e', 't', 'h', 'f', 'j', 'z', 'p', 'k', 'w', 'x', 'm', 'a', 'y']
{'s': 0, 'i': 1, 'l': 2, 'n': 3, 'b': 4, 'v': 5, 'r': 6, 'o': 7, 'd': 8, ' ': 9, 'q': 10, 'g': 11, 'c': 12, 'u': 13, 'e': 14, 't': 15, 'h': 16, 'f': 17, 'j': 18, 'z': 19, 'p': 20, 'k': 21, 'w': 22, 'x': 23, 'm': 24, 'a': 25, 'y': 26}
5032359


Lastly we import other useful libraries to help you getting started.

In [41]:
import d2l
import math
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, rnn
import time

## 1. Train Recurrent Latent Variable Models

Train a number of different latent variable models using `train_indices` to assess their performance. By default pick 256 dimensions for the hidden units. You can use the codes provided in the class. Also, we strongly encourage you to use the Gluon implementation since it's a lot faster than building it from scratch.

1. Train a single-layer RNN (with latent variables). 
1. Train a single-layer GRU.
1. Train a single-layer LSTM. 
1. Train a two-layer LSTM. 

How low can you drive the perplexity? Can you reproduce some of Shakespeare's finest writing (generate 200 characters). Start the sequence generator with `But Brutus is an honorable man`. Experiment with a number of settings:

* Number of hidden units. 
* Embedding length.
* Gradient clipping.
* Number of iterations. 
* Learning rate.

**Save** the models (at least in memory since you'll need them in the next exercise.

In [4]:
num_hiddens = 256
rnn_layer = rnn.RNN(num_hiddens)
rnn_layer.initialize()

In [5]:
batch_size = 2
state = rnn_layer.begin_state(batch_size=batch_size)
state[0].shape

(1, 2, 256)

In [ ]:
gru_layer = rnn.GRU(num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size)
ctx = d2l.try_gpu()
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['but brutus is an honorable man']

In [10]:
d2l.train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
                               corpus_indices, idx_to_char, char_to_idx,
                               num_epochs, num_steps, lr, clipping_theta,
                               batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 3.543577, time 58.54 sec


KeyError: 'B'

## 2. Test Error

So far we measured perplexity only on the training set. 

1. Implement a perplexity calculator that does not involve training.
1. Compute the perplexity of the best models in each of the 4 categories on the test set. By how much does it differ? 

## 3. $N$-Gram Model

So far we only considered latent variable models. Let's see what happens if we use a regular $N$-gram model and an autoregressive setting. That is, we aim to predict the next character given the current characters one character at a time. For this implement the following:

1. Split data into $(x,y)$ pairs as before, just that we now use very short subsequences, e.g. only $5$ characters. That is, `But Brutus` turns into the tuples (`(But B, r)`, `(ut Br, u)`, `(t Bru, t)`, `( Brut, u)`, `(Brutu, s)`). 
1. Use one-hot encoding for each character separately and combine them all. 
    * In one case use a sequential encoding to obtain an embedding proportional to the length of the sequence.
    * Use a bag of characters encoding that sums over all occurrences.
1. Implement an MLP with one hidden layer and 256 hidden units.
1. Train it to output the next character.

How accurate is the model? How does the number of operations and weights compare to an RNN, a GRU and an LSTM discussed above?

In [82]:
raw_dataset = 'But Brutus'
idx_to_char = list(set(raw_dataset))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
corpus_indices = [char_to_idx[char] for char in raw_dataset]
#sample = corpus_indices[:20]
#print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
#print('indices:', sample)

#train_indices = corpus_indices[:-100000]
#test_indices = corpus_indices[-100000:]

## Data splitting

In [71]:
embedding = 5 # embedding dimension for autoregressive model
T = len(raw_dataset)
features = nd.zeros((T-embedding, embedding))

In [72]:
for i in range(embedding):
    features[:,i] = corpus_indices[i:T-embedding+i]
labels = corpus_indices[embedding:]

In [73]:
features


[[20.  6.  7. 18. 14.]
 [ 6.  7. 18. 14. 12.]
 [ 7. 18. 14. 12. 15.]
 ...
 [22.  9. 14.  4.  7.]
 [ 9. 14.  4.  7.  7.]
 [14.  4.  7.  7. 21.]]
<NDArray 5032354x5 @cpu(0)>

### example

In [83]:
embedding = 5 # embedding dimension for autoregressive model
T = len(raw_dataset)
features = nd.zeros((T-embedding, embedding))

In [84]:
for i in range(embedding):
    features[:,i] = corpus_indices[i:T-embedding+i]
labels = corpus_indices[embedding:]

In [101]:
features.shape

(5, 5)

## Encoding

In [117]:
def to_onehot_length(X, vocab_size):
    inputs = [nd.one_hot(x, vocab_size) for x in X]
    matrix_size = inputs[0].size
    for i in range(len(inputs)):
        inputs[i] = inputs[i].reshape(1,matrix_size)
    return inputs

In [142]:
def to_onehot_sum(X, vocab_size):
    inputs = [nd.one_hot(x, vocab_size) for x in X]
    matrix_size = inputs[0].size
    for i in range(len(inputs)):
        inputs[i] = nd.sum(inputs[i],axis = 0).reshape(1, vocab_size)
    return inputs

In [118]:
inputs = to_onehot_length(features, vocab_size)

In [143]:
inputs = to_onehot_sum(features, vocab_size)
inputs

[
 [[1. 0. 1. 2. 1. 0.]]
 <NDArray 1x6 @cpu(0)>, 
 [[1. 0. 1. 1. 1. 1.]]
 <NDArray 1x6 @cpu(0)>, 
 [[1. 0. 1. 1. 1. 1.]]
 <NDArray 1x6 @cpu(0)>, 
 [[1. 0. 1. 1. 1. 1.]]
 <NDArray 1x6 @cpu(0)>, 
 [[0. 0. 2. 1. 1. 1.]]
 <NDArray 1x6 @cpu(0)>]

### MLP

In [48]:
net = gluon.nn.Sequential()
net.add(gluon.nn.Dense(256, activation='relu'))
net.add(gluon.nn.Dense(vocab_size))
net.initialize(init.Xavier())

loss = gloss.SoftmaxCrossEntropyLoss()

In [49]:
ntrain = 4932354
train_data = gluon.data.ArrayDataset(features[:ntrain,:], labels[:ntrain])
test_data  = gluon.data.ArrayDataset(features[ntrain:,:], labels[ntrain:])

batch_size = 30
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.5})
num_epochs = 10

train_iter = gluon.data.DataLoader(train_data, batch_size, shuffle=True)
test_iter = gluon.data.DataLoader(test_data, batch_size, shuffle=True)

In [ ]:
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None,
              None, trainer)